# LIBRARY BLOCK

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from bs4 import BeautifulSoup
import pandas as pd
import time 
from datetime import datetime
import re


# SET UP BLOCK

# FUNCTION BLOCK

In [2]:

def load_full_content(max_clicks=10):
    # Cấu hình số lần click tối đa và bộ đếm
    current_click = 0

    while current_click < max_clicks:
        print(f"--- Bắt đầu vòng lặp thứ {current_click + 1} ---")
        
        # === BƯỚC 1: SCROLL CHO ĐẾN KHI KHÔNG TẢI THÊM NỘI DUNG ===
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            # Scroll xuống cuối trang
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            # Chờ trang tải nội dung (2 giây)
            time.sleep(2)
            
            # Tính toán chiều cao mới của trang sau khi scroll
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            if new_height == last_height:
                # Nếu chiều cao không đổi, nghĩa là đã chạm đáy
                print("Đã scroll đến cuối nội dung hiện tại.")
                break
            
            last_height = new_height

        # === BƯỚC 2: CLICK NÚT NEXT/VIEW MORE ===
        try:
            # Tìm lại nút next mỗi lần lặp để tránh lỗi "Stale Element Reference"
            next_button = driver.find_element(By.CSS_SELECTOR, 'div[class="btn-viewmore"]')
            
            if next_button.is_displayed():
                # Sử dụng Javascript để click (tránh trường hợp nút bị che bởi quảng cáo hoặc header)
                driver.execute_script("arguments[0].click();", next_button)
                
                current_click += 1
                print(f"Đã click nút Next lần thứ: {current_click}")
                
                # Yêu cầu: Chờ 10 giây sau mỗi lần click
                time.sleep(10)
            else:
                print("Nút Next không hiển thị. Dừng vòng lặp.")
                break
                
        except NoSuchElementException:
            print("Không tìm thấy nút Next (có thể đã hết trang). Dừng vòng lặp.")
            break
        except Exception as e:
            print(f"Có lỗi xảy ra: {e}")
            

# def get_generalization_news_data():
#     # === BƯỚC 3: LẤY DỮ LIỆU SAU KHI ĐÃ LOAD HẾT ===
#     print("--- Hoàn tất quá trình load. Bắt đầu lấy dữ liệu ---")

#     box_news = driver.find_elements(By.CSS_SELECTOR, 'div[class="list-news-main top5_news"] div[class="tlitem box-category-item"]')
#     print(f"Tổng số tin tìm thấy: {len(box_news)}")
#     if len(box_news) == 0:
#         box_news = driver.find_elements(By.CSS_SELECTOR, 'div[class="tlitem box-category-item"]')
#     print(f"Tổng số tin tìm thấy sau khi thử selector khác: {len(box_news)}")   
        

#     data = []
#     for idx,box in enumerate(box_news):   
#         title = box.find_element(By.CSS_SELECTOR, 'h3 a').text
#         link = box.find_element(By.CSS_SELECTOR, 'h3 a').get_attribute('href')
#         time_post = box.find_element(By.CSS_SELECTOR, 'span[class="time time-ago"]').get_attribute('title')
#         summary = box.find_element(By.CSS_SELECTOR, 'p[class="sapo box-category-sapo"]').text
#         print(f"Đang xử lý tin thứ {idx + 1}")
#         print(title)
#         data.append({
#             "title": title,
#             "link": link,
#             "time_post": time_post,
#             "summary": summary
#         })
#     return data
def get_generalization_news_data():
    print("--- Hoàn tất quá trình load. Bắt đầu lấy dữ liệu (Batch Mode) ---")

    # 1. Xác định container chính để giới hạn phạm vi tìm kiếm (tránh lấy nhầm tin ở footer/sidebar)
    # Tìm vùng chứa tin
    try:
        container = driver.find_element(By.CSS_SELECTOR, 'div[class="list-news-main top5_news"]')
    except:
        container = driver # Nếu không thấy thì tìm trên toàn trang (fallback)

    # 2. Lấy TOÀN BỘ từng thuộc tính ra các danh sách riêng biệt
    # Lưu ý: Phải dùng selector trỏ thẳng vào element con cuối cùng
    list_titles_els = container.find_elements(By.CSS_SELECTOR, 'div[class="tlitem box-category-item"] h3 a')
    list_time_els = container.find_elements(By.CSS_SELECTOR, 'div[class="tlitem box-category-item"] span[class="time time-ago"]')
    list_summary_els = container.find_elements(By.CSS_SELECTOR, 'div[class="tlitem box-category-item"] p[class="sapo box-category-sapo"]')

    # 3. KIỂM TRA SỐ LƯỢNG (Bước quan trọng bạn đã nhắc đến)
    count_title = len(list_titles_els)
    count_time = len(list_time_els)
    count_summary = len(list_summary_els)

    print(f"Check số lượng: Title={count_title}, Time={count_time}, Summary={count_summary}")

    data = []

    # TRƯỜNG HỢP 1: Dữ liệu hoàn hảo (Số lượng bằng nhau hết) -> Ghép nhanh
    if count_title == count_time == count_summary:
        print("-> Dữ liệu đồng bộ. Đang ghép nhanh...")
        # Hàm zip giúp gộp 3 danh sách lại chạy song song
        count = 0   
        for title_el, time_el, summary_el in zip(list_titles_els, list_time_els, list_summary_els):
            count += 1
            print(f"Đang xử lý tin thứ {count + 1}: {title_el.text}")
            data.append({
                "title": title_el.text,
                "link": title_el.get_attribute('href'),
                "time_post": time_el.get_attribute('title'),
                "summary": summary_el.text
            })

    # TRƯỜNG HỢP 2: Dữ liệu bị lệch (Có tin thiếu time hoặc summary) -> Phải chạy chậm lại để an toàn
    else:
        print("-> CẢNH BÁO: Số lượng không khớp (có tin thiếu nội dung). Chuyển về chế độ quét từng dòng (chậm nhưng an toàn).")
        # Quay lại cách cũ: Tìm box cha rồi tìm con
        boxes = container.find_elements(By.CSS_SELECTOR, 'div[class="tlitem box-category-item"]')
        for box in boxes:
            try:
                # Dùng try-except cho từng element con để nếu thiếu cũng không lỗi code
                t_el = box.find_element(By.CSS_SELECTOR, 'h3 a')
                title = t_el.text
                link = t_el.get_attribute('href')
            except: 
                title = ""; link = ""
            
            try:
                time_post = box.find_element(By.CSS_SELECTOR, 'span[class="time time-ago"]').get_attribute('title')
            except:
                time_post = ""

            try:
                summary = box.find_element(By.CSS_SELECTOR, 'p[class="sapo box-category-sapo"]').text
            except:
                summary = ""

            print(f"Đang xử lý tin: {title}")
            data.append({
                "title": title, "link": link, "time_post": time_post, "summary": summary
            })

    return data

def get_detail_news_data(df):
    data = []
    
    for i in df.index:
        link = df.loc[i, 'link']
        title_ref = df.loc[i, 'title']
        
        # 1. Truy cập trang
        try:
            driver.get(link)
        except Exception as e:
            print(f"Lỗi không truy cập được link bài {i+1}: {e}")
            # Nếu link chết, append data rỗng và continue
            data.append({
                "title": title_ref, "content": "", "link": link,
                "time_post": df.loc[i, 'time_post'], "summary": df.loc[i, 'summary']
            })
            continue

        print(f"--- Đang xử lý bài báo thứ {i + 1} ---")
        
        # 2. Cơ chế chờ thông minh (Tối đa 3 phút)
        start_time = time.time()
        content = "" # Mặc định là rỗng
        got_data = False # Cờ đánh dấu đã lấy được chưa

        while (time.time() - start_time) < 180: # 180 giây = 3 phút
            try:
                # Cố gắng tìm container chứa bài viết trước
                # Lưu ý: Cần kiểm tra kỹ selector 'div[class="w640 fr clear"]' có đúng cho mọi bài không
                main_div = driver.find_element(By.CSS_SELECTOR, 'div[class="w640 fr clear"]')
                
                # Lấy H2 (Sapo/Mô tả)
                # Dùng try-except nhỏ ở đây vì có thể có bài không có h2
                try:
                    h2_text = main_div.find_element(By.TAG_NAME, 'h2').text
                except NoSuchElementException:
                    h2_text = ""

                # Lấy các thẻ P (Nội dung chi tiết)
                content_elements = main_div.find_elements(By.TAG_NAME, 'p')

                # ĐIỀU KIỆN ĐỂ COI LÀ THÀNH CÔNG:
                # Phải tìm thấy ít nhất 1 thẻ P hoặc có H2
                if len(content_elements) > 0 or h2_text != "":
                    # Ghép chuỗi
                    temp_content = h2_text + "\n"
                    for p in content_elements:
                        temp_content += p.text + "\n"
                    
                    content = temp_content
                    got_data = True
                    print(f"    -> Đã lấy xong nội dung ({len(content)} ký tự).")
                    break # Thoát vòng lặp while ngay lập tức
                
            except (NoSuchElementException, StaleElementReferenceException):
                # Nếu chưa tìm thấy element cha, hoặc trang đang reload DOM
                pass
            except Exception as e:
                print(f"    Lỗi không xác định trong lúc chờ: {e}")
            
            # Nếu chưa lấy được, nghỉ 1 giây rồi thử lại
            time.sleep(1)

        # 3. Kiểm tra kết quả sau vòng lặp
        if not got_data:
            print(f"    -> CẢNH BÁO: Quá 3 phút không lấy được nội dung bài {i+1}. Để trống content.")
            content = "" # Đảm bảo rỗng như yêu cầu

        # 4. Lưu vào danh sách
        data.append({
            "title": title_ref,
            "content": content,
            "link": link,
            "time_post": df.loc[i, 'time_post'],
            "summary": df.loc[i, 'summary']
        })
        
    return data

# EXECUTION BLOCK

In [3]:
global driver
driver = webdriver.Chrome()
driver.get("https://cafef.vn/thi-truong-chung-khoan.chn")

In [13]:
load_full_content(max_clicks=100)

--- Bắt đầu vòng lặp thứ 1 ---
Đã scroll đến cuối nội dung hiện tại.
Đã click nút Next lần thứ: 1
--- Bắt đầu vòng lặp thứ 2 ---
Đã scroll đến cuối nội dung hiện tại.
Đã click nút Next lần thứ: 2
--- Bắt đầu vòng lặp thứ 3 ---
Đã scroll đến cuối nội dung hiện tại.
Đã click nút Next lần thứ: 3
--- Bắt đầu vòng lặp thứ 4 ---
Đã scroll đến cuối nội dung hiện tại.
Đã click nút Next lần thứ: 4
--- Bắt đầu vòng lặp thứ 5 ---
Đã scroll đến cuối nội dung hiện tại.
Đã click nút Next lần thứ: 5
--- Bắt đầu vòng lặp thứ 6 ---
Đã scroll đến cuối nội dung hiện tại.
Đã click nút Next lần thứ: 6
--- Bắt đầu vòng lặp thứ 7 ---
Đã scroll đến cuối nội dung hiện tại.
Đã click nút Next lần thứ: 7
--- Bắt đầu vòng lặp thứ 8 ---
Đã scroll đến cuối nội dung hiện tại.
Đã click nút Next lần thứ: 8
--- Bắt đầu vòng lặp thứ 9 ---
Đã scroll đến cuối nội dung hiện tại.
Đã click nút Next lần thứ: 9
--- Bắt đầu vòng lặp thứ 10 ---
Đã scroll đến cuối nội dung hiện tại.
Đã click nút Next lần thứ: 10
--- Bắt đầu vòng l

In [45]:
data = get_generalization_news_data()
df = pd.DataFrame(data)
# Lưu vào file có tên là 'du_lieu_tin_tuc.json'
df.to_json(
    'data/danh_sach_bai_bao.json', 
    orient='records',    # Lưu dưới dạng danh sách các đối tượng: [{}, {}, ...]
    force_ascii=False,   # QUAN TRỌNG: Để giữ nguyên tiếng Việt (không bị chuyển thành mã \uXXXX)
    indent=4             # Thụt đầu dòng để file dễ đọc hơn (pretty print)
)

print("Đã lưu file danh_sach_bai_bao.json thành công!")

--- Hoàn tất quá trình load. Bắt đầu lấy dữ liệu (Batch Mode) ---
Check số lượng: Title=907, Time=907, Summary=907
-> Dữ liệu đồng bộ. Đang ghép nhanh...
Đang xử lý tin thứ 2: NSH Petro có Tổng Giám đốc mới sau gần 2 tháng để ‘trống ghế’
Đang xử lý tin thứ 3: Quỹ ETF ngoại quy mô 15.000 tỷ thêm mới TCX, dự kiến bán mạnh VIC
Đang xử lý tin thứ 4: Viglacera miễn nhiệm một Phó Tổng Giám đốc
Đang xử lý tin thứ 5: Chuyên gia lý giải nguyên nhân chứng khoán Việt Nam "sale off" 52 điểm ngày 12-12
Đang xử lý tin thứ 6: Một thế lực bất ngờ tung hàng trăm tỷ "gom" cổ phiếu Việt Nam giữa lúc thị trường rơi mạnh trong phiên cuối tuần
Đang xử lý tin thứ 7: Chứng khoán trượt mạnh
Đang xử lý tin thứ 8: Vì sao VN-Index “bốc hơi” 52 điểm trong phiên cuối tuần?
Đang xử lý tin thứ 9: Coteccons chốt ngày phát hành hơn 5 triệu cổ phiếu thưởng cho cổ đông
Đang xử lý tin thứ 10: Phiên 12/12: Khối ngoại tiếp đà bán ròng hơn 500 tỷ đồng, cổ phiếu nào bị "xả" mạnh nhất?
Đang xử lý tin thứ 11: Chứng khoán Việt N

In [66]:
df = pd.read_json('data/danh_sach_bai_bao.json')
data = get_detail_news_data(df)
df_data = pd.DataFrame(data)
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
df_data.to_json(
    f'data/newspaper_data/chi_tiet_bai_bao_{current_time}.json',
    orient='records',
    force_ascii=False,
    indent=4
)


--- Đang xử lý bài báo thứ 1 ---
    -> Đã lấy xong nội dung (2309 ký tự).
--- Đang xử lý bài báo thứ 2 ---
    -> Đã lấy xong nội dung (2230 ký tự).
--- Đang xử lý bài báo thứ 3 ---
    -> Đã lấy xong nội dung (2580 ký tự).
--- Đang xử lý bài báo thứ 4 ---
    -> Đã lấy xong nội dung (5859 ký tự).
--- Đang xử lý bài báo thứ 5 ---
    -> Đã lấy xong nội dung (1211 ký tự).
--- Đang xử lý bài báo thứ 6 ---
    -> Đã lấy xong nội dung (2386 ký tự).
--- Đang xử lý bài báo thứ 7 ---
    -> Đã lấy xong nội dung (3681 ký tự).
--- Đang xử lý bài báo thứ 8 ---
    -> Đã lấy xong nội dung (3038 ký tự).
--- Đang xử lý bài báo thứ 9 ---
    -> Đã lấy xong nội dung (2333 ký tự).
--- Đang xử lý bài báo thứ 10 ---
    -> Đã lấy xong nội dung (3775 ký tự).
--- Đang xử lý bài báo thứ 11 ---
    -> Đã lấy xong nội dung (4482 ký tự).
--- Đang xử lý bài báo thứ 12 ---
    -> Đã lấy xong nội dung (5116 ký tự).
--- Đang xử lý bài báo thứ 13 ---
    -> Đã lấy xong nội dung (2236 ký tự).
--- Đang xử lý bài bá